In [1]:
import h5py, os, torch
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, Dataset
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

## Outline
 - Generate a specific model for each scene
     - Each location: 1600 images w/o presence, 800 imags w/ presence
 - Create Dataloader
     - 80-20 split for training/testing (validation?)
     - Each img (input size): 1024 x 720 px
 - Model Architecture
     - Modified VGG16?
     - Custom CNN architecture?
     - Concatenate pixel values and put in logistic regression?

---

## Model Base/Helpers/Train & Test functions

In [3]:
def accuracy(outs, labels):
    # check accuracy for threshold @ 0.7
    outs_th = outs >= 0.7
        # append onto result dictionary to be returned to function that called `accuracy()`
    return torch.tensor(torch.sum(outs_th == labels).item() / len(outs))

In [4]:
class ModelBase(nn.Module):
#     training step
    def train_step(self, batch):
        xb, labels = batch
        labels = labels.view(-1, 1)
        outs = self(xb)
        loss = F.binary_cross_entropy(outs, labels)
        return loss
#     validation step
    def val_step(self, batch):
        xb, labels = batch
        labels = labels.view(-1, 1)
        outs = self(xb)
        loss = F.binary_cross_entropy(outs, labels)
        acc = accuracy(outs, labels)
        return {'loss': loss.detach(), 'acc': acc}
#     validation epoch (avg accuracies and losses)
    def val_epoch_end(self, outputs):
#         for i in range(10):
#             print(outputs[i])
        batch_loss = [x['loss'] for x in outputs]
        avg_loss = torch.stack(batch_loss).mean()
        batch_acc = [x['acc'] for x in outputs]
        avg_acc = torch.stack(batch_acc).mean()
        return {'avg_loss': avg_loss, 'avg_acc': avg_acc}
#     print everything important
    def epoch_end(self, epoch, avgs, test=False):
        s = 'test' if test else 'val'
        print(f'EPOCH {epoch + 1:<10} | {s}_loss:{avgs["avg_loss"]:.3f}, {s}_acc (0.7 thr): {avgs["avg_acc"]:.3f}')

In [5]:
@torch.no_grad()
def evaluate(model, val_dl):
    # eval mode
    model.eval()
    outputs = [model.val_step(batch) for batch in val_dl]
    return model.val_epoch_end(outputs)


def fit(epochs, lr, model, train_dl, val_dl, opt_func=torch.optim.Adam):
    torch.cuda.empty_cache()
    history = []
    # define optimizer
    optimizer = opt_func(model.parameters(), lr)
    # for each epoch...
    for epoch in range(epochs):
        # training mode
        model.train()
        # (training) for each batch in train_dl...
        for batch in tqdm(train_dl):
            # pass thru model
            loss = model.train_step(batch)
            # perform gradient descent
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # validation
        res = evaluate(model, val_dl)
        # print everything useful
        model.epoch_end(epoch, res, test=False)
        # append to history
        history.append(res)
    return history

---

## Models

In [6]:
class VGG11_PT(ModelBase):
    def __init__(self):
        super().__init__()
#         pretrained VGG16 model w/ batch norm
        self.network = torchvision.models.vgg11_bn(pretrained=True)
#         change first layer to accept only 1 dimension of color (b/w)
        self.network.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
#         change last fc layer to output one value
        self.network.classifier[6] = nn.Linear(4096, 1, bias=True)
        
    def forward(self, xb):
        out = self.network(xb)
        out = F.sigmoid(out)
        return out

In [7]:
class Custom_NPT(ModelBase):
    def __init__(self):
        super().__init__()                                       # 1 x 1024 x 720
#         custom-defined model w/o pretrained weights
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)   # 8 x 1024 x 720
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)  # 16 x 1024 x 720
        self.pool1 = nn.MaxPool2d(2)                             # 16 x 512 x 360
        self.conv3 = nn.Conv2d(16, 16, kernel_size=3, padding=1) # 16 x 512 x 360
        self.pool2 = nn.MaxPool2d(2)                             # 16 x 256 x 180
        self.conv5 = nn.Conv2d(16, 8, kernel_size=3, padding=1)  # 8 x 256 x 180
        self.pool3 = nn.MaxPool2d(2)                             # 8 x 128 x 90
        self.fc1 = nn.Linear(8*128*90, 4096)
        self.fc2 = nn.Linear(4096, 512)
        self.fc3 = nn.Linear(512, 1)
        
    def forward(self, xb):
        out = F.relu(self.conv1(xb))
        out = F.relu(self.conv2(out))
        out = self.pool1(out)
        out = F.relu(self.conv3(out))
        out = self.pool2(out)
        out = F.relu(self.conv5(out))
        out = self.pool3(out)

        out = torch.flatten(out, 1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        out = F.sigmoid(out)
        return out

---

## DataSet & DataLoader

In [8]:
class Prudhoe_DS(Dataset):
    def __init__(self, npy_pth, transforms=None):
        self.transforms = transforms
        self.npy_pth = npy_pth
        self.masks_loc = []
        self.labels = []
        
        for typ in ['pos', 'neg']:
            for fn in os.listdir(os.path.join(self.npy_pth, typ)):
                
                full_fn = os.path.join(self.npy_pth, typ, fn)
                self.masks_loc.append(full_fn)
#                 masks.append(np.load(full_fn))
                
                label = 1 if typ == 'pos' else 0
                self.labels.append(label)
        
        self.masks_loc = np.array(self.masks_loc)
        self.labels = np.array(self.labels).astype('float32')
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        mask = np.expand_dims(np.load(self.masks_loc[idx]), axis=0).astype('float32')
        label = self.labels[idx]
        
        if self.transforms:
            mask = self.transforms()(mask)
        
        return mask, label

---

In [9]:
torch.manual_seed(42)

In [10]:
PTH = '/scratch/richardso21/20-21_BGSUB/'
site = 'prudhoe_12'

In [11]:
npy_pth = os.path.join(PTH, 'FgSegNet_O_neg', site)

In [12]:
ds = Prudhoe_DS(npy_pth)
ds_size = len(ds)

In [13]:
train_size = int(ds_size * .70)
val_size = (ds_size - train_size) // 2
test_size = ds_size - train_size - val_size

In [14]:
train_ds, val_ds, test_ds = random_split(ds, [train_size, val_size, test_size])

In [15]:
batch_size = 4

In [16]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size*2, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_ds, batch_size*2, pin_memory=True, num_workers=4)

---

In [17]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [18]:
device = get_default_device()
device

device(type='cuda')

In [19]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [20]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

In [21]:
model = to_device(Custom_NPT(), device)

In [22]:
history = [evaluate(model, val_loader)]
history

/scratch/richardso21/conda/envs/fgsegnet/lib/python3.6/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[{'avg_loss': tensor(0.6964, device='cuda:0'), 'avg_acc': tensor(0.6273)}]

In [23]:
history += fit(20, 1e-4, model, train_loader, val_loader)

  0%|          | 0/496 [00:00<?, ?it/s]

EPOCH 1          | val_loss:0.662, val_acc (0.7 thr): 0.627


  0%|          | 0/496 [00:00<?, ?it/s]

EPOCH 2          | val_loss:0.658, val_acc (0.7 thr): 0.630


  0%|          | 0/496 [00:00<?, ?it/s]

...

EPOCH 18         | val_loss:1.714, val_acc (0.7 thr): 0.801


  0%|          | 0/496 [00:00<?, ?it/s]

EPOCH 19         | val_loss:1.955, val_acc (0.7 thr): 0.854


100%|██████████| 496/496 [00:49<00:00, 10.01it/s]


EPOCH 20         | val_loss:1.575, val_acc (0.7 thr): 0.850


Loss increases, but so does accuracy ???
probably diverging (learning rate too high / didn't use batch norm)

In [24]:
evl = evaluate(model, test_loader)
evl

{'avg_loss': tensor(1.5038, device='cuda:0'), 'avg_acc': tensor(0.8472)}

In [27]:
history += fit(10, 1e-6, model, train_loader, val_loader)

  0%|          | 0/496 [00:00<?, ?it/s]

EPOCH 1          | val_loss:1.762, val_acc (0.7 thr): 0.861


  0%|          | 0/496 [00:00<?, ?it/s]

EPOCH 2          | val_loss:1.567, val_acc (0.7 thr): 0.866


  0%|          | 0/496 [00:00<?, ?it/s]

...

EPOCH 8          | val_loss:1.564, val_acc (0.7 thr): 0.868


  0%|          | 0/496 [00:00<?, ?it/s]

EPOCH 9          | val_loss:1.564, val_acc (0.7 thr): 0.868


100%|██████████| 496/496 [00:49<00:00, 10.03it/s]


EPOCH 10         | val_loss:1.564, val_acc (0.7 thr): 0.870


In [57]:
evl = evaluate(model, test_loader)
evl

{'avg_loss': tensor(1.2770, device='cuda:0'), 'avg_acc': tensor(0.8750)}

In [28]:
torch.save(model.state_dict(), '/scratch/richardso21/20-21_BGSUB/CustomNPT12_2.pth')

In [26]:
# torch.cuda.empty_cache()

In [47]:
x = torch.tensor([1,2,3], device=device)

In [48]:
y = [x]

In [49]:
y

[tensor([1, 2, 3], device='cuda:0')]

In [50]:
np.save('/scratch/richardso21/20-21_BGSUB/test3.npy', np.array(y))

In [107]:
model.eval()
allOuts = np.array([])
allLabels = np.array([])
for batch in test_loader:
    xb, labels = batch
    outs = model(xb).data.view(-1)
    allOuts = np.append(allOuts, outs.cpu().numpy())
    allLabels = np.append(allLabels, labels.cpu().numpy())

In [108]:
allLabels.shape

(425,)

In [109]:
allOuts.shape

(425,)

In [69]:
a = np.array([])

In [75]:
a = np.append(a, [1,2,3])

In [77]:
a = np.append(a, [1])

In [78]:
a

array([1., 2., 3., 1.])